"2024-05-29-Dill Picking Errors with simple ComputationResult classes.ipynb"
based off of "2024-01-02-Dill Picking Error with QApplication.ipynb"


# Problem: Simple `ComputedResult` subclasses cannot seem to be pickled!

# Solution: The problem strangely resolved itself from last night. All pickle correctly after removing the `super(...).init(...)` call in my `.__setstate__()` func`

In [ ]:
# Observation: Sometimes specific result can be successfully picked while entire pipeline cannot:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import save_rank_order_results, SaveStringGenerator
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.core import Epoch
from nptyping import NDArray
import neuropy.utils.type_aliases as types

import numpy as np
import pandas as pd
from attrs import define, field, Factory, asdict, astuple
from functools import wraps
from copy import deepcopy
from collections import namedtuple
from pathlib import Path
from datetime import datetime, date, timedelta
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, custom_define, serialized_field, serialized_attribute_field, non_serialized_field, keys_only_repr

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData
from pyphoplacecellanalysis.General.Model.ComputationResults import ComputedResult



# ==================================================================================================================== #
# 2024-05-28 - TrialByTrialActivity                                                                                    #
# ==================================================================================================================== #
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent
from pyphoplacecellanalysis.Analysis.reliability import TrialByTrialActivity


@define(slots=False, repr=False, eq=False)
class TrialByTrialActivityResult(ComputedResult):
    """ 


    #TODO 2024-05-28 19:14: - [ ] Not yet picklable, and I think it just needs a recurrsive __getstate__(...) function

    
    Usage:
    
        from pyphoplacecellanalysis.Analysis.reliability import TrialByTrialActivity
        from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrialByTrialActivityResult

        directional_trial_by_trial_activity_result: TrialByTrialActivityResult = curr_active_pipeline.global_computation_results.computed_data.get('TrialByTrialActivity', None)
        any_decoder_neuron_IDs = directional_trial_by_trial_activity_result.any_decoder_neuron_IDs
        active_pf_dt: PfND_TimeDependent = directional_trial_by_trial_activity_result.active_pf_dt
        directional_lap_epochs_dict: Dict[str, Epoch] = directional_trial_by_trial_activity_result.directional_lap_epochs_dict
        directional_active_lap_pf_results_dicts: Dict[str, TrialByTrialActivity] = directional_trial_by_trial_activity_result.directional_active_lap_pf_results_dicts
        directional_active_lap_pf_results_dicts

    """
    _VersionedResultMixin_version: str = "2024.05.28_0" # to be updated in your IMPLEMENTOR to indicate its version

    any_decoder_neuron_IDs: NDArray = serialized_field(default=None)
    active_pf_dt: PfND_TimeDependent = serialized_field(default=None)
    directional_lap_epochs_dict: Dict[str, Epoch] =  serialized_field(default=None)
    directional_active_lap_pf_results_dicts: Dict[str, TrialByTrialActivity] = serialized_field(default=None)

    def __repr__(self):
        """ 2024-01-11 - Renders only the fields and their sizes
        """
        from pyphocorehelpers.print_helpers import strip_type_str_to_classname
        attr_reprs = []
        for a in self.__attrs_attrs__:
            attr_type = strip_type_str_to_classname(type(getattr(self, a.name)))
            if 'shape' in a.metadata:
                shape = ', '.join(a.metadata['shape'])  # this joins tuple elements with a comma, creating a string without quotes
                attr_reprs.append(f"{a.name}: {attr_type} | shape ({shape})")  # enclose the shape string with parentheses
            else:
                attr_reprs.append(f"{a.name}: {attr_type}")
        content = ",\n\t".join(attr_reprs)
        return f"{type(self).__name__}({content}\n)"
    


In [ ]:
a_trial_by_trial_result: TrialByTrialActivityResult = TrialByTrialActivityResult(is_global=True)
a_trial_by_trial_result

In [ ]:
type(a_trial_by_trial_result)

In [ ]:
saveData('test_a_trial_by_trial_result_data.pkl', a_trial_by_trial_result)


In [ ]:


# spikes_df = curr_active_pipeline.sess.spikes_df
rank_order_results = global_computation_results.computed_data['RankOrder'] # : "RankOrderComputationsContainer"
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
# included_qclu_values: List[int] = rank_order_results.included_qclu_values
directional_laps_results: DirectionalLapsResult = global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only -- !! Is minimum_inclusion_fr_Hz=None the issue/difference?
# long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']
# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [owning_pipeline_reference.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also

## INPUTS: curr_active_pipeline, track_templates, global_epoch_name, (long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj)
any_decoder_neuron_IDs: NDArray = deepcopy(track_templates.any_decoder_neuron_IDs)
long_epoch_name, short_epoch_name, global_epoch_name = owning_pipeline_reference.find_LongShortGlobal_epoch_names()

# ## Directional Trial-by-Trial Activity:
if 'pf1D_dt' not in owning_pipeline_reference.computation_results[global_epoch_name].computed_data:
    # if `KeyError: 'pf1D_dt'` recompute
    owning_pipeline_reference.perform_specific_computation(computation_functions_name_includelist=['pfdt_computation'], enabled_filter_names=None, fail_on_exception=True, debug_print=False)

active_pf_1D_dt: PfND_TimeDependent = deepcopy(owning_pipeline_reference.computation_results[global_epoch_name].computed_data['pf1D_dt'])
# active_pf_2D_dt: PfND_TimeDependent = deepcopy(owning_pipeline_reference.computation_results[global_epoch_name].computed_data['pf2D_dt'])

active_pf_dt: PfND_TimeDependent = active_pf_1D_dt
# Limit only to the placefield aclus:
active_pf_dt = active_pf_dt.get_by_id(ids=any_decoder_neuron_IDs)

# active_pf_dt: PfND_TimeDependent = deepcopy(active_pf_2D_dt) # 2D
long_LR_name, long_RL_name, short_LR_name, short_RL_name = track_templates.get_decoder_names()

directional_lap_epochs_dict = dict(zip((long_LR_name, long_RL_name, short_LR_name, short_RL_name), (long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj)))
directional_active_lap_pf_results_dicts: Dict[str, TrialByTrialActivity] = TrialByTrialActivity.directional_compute_trial_by_trial_correlation_matrix(active_pf_dt=active_pf_dt, directional_lap_epochs_dict=directional_lap_epochs_dict, included_neuron_IDs=any_decoder_neuron_IDs)

## OUTPUTS: directional_active_lap_pf_results_dicts
a_train_test_result: TrialByTrialActivityResult = TrialByTrialActivityResult(any_decoder_neuron_IDs=any_decoder_neuron_IDs,
                                                                                active_pf_dt=active_pf_dt,
                                                                                directional_lap_epochs_dict=directional_lap_epochs_dict,
                                                                                directional_active_lap_pf_results_dicts=directional_active_lap_pf_results_dicts,
                                                                                is_global=True)  # type: Tuple[Tuple[Dict[str, Any], Dict[str, Any]], Dict[str, BasePositionDecoder], Any]

In [ ]:
# 2023-01-02 - Fails SOMETIMES, even when called immediately after the above save command which worked:
curr_active_pipeline.save_global_computation_results()

# Clearing and reloading the notebook, skipping these computations, and then re-applying them to the pipeline does allow resaving with `curr_active_pipeline.save_global_computation_results()` for some reason.



In [ ]:
curr_active_pipeline.global_computation_results.computed_data['RankOrder']

import dill as pickle

def diagnose_pickling_issues(object_to_pickle):
   """Intellegently diagnoses which property on an object is causing pickling via Dill to fail."""

   try:
       # Attempt to pickle the object directly
       pickle.dumps(object_to_pickle)
   except pickle.PicklingError as e:
       # If pickling fails, initiate a diagnostic process
       print(f"Pickling error encountered: {e}")

       # Gather information about the object's attributes
       object_attributes = [attr for attr in dir(object_to_pickle) if not attr.startswith("__")]

       # Isolate problematic attributes through iterative testing
       problematic_attribute = None
       for attribute in object_attributes:
           try:
               pickle.dumps(getattr(object_to_pickle, attribute))
           except pickle.PicklingError:
               problematic_attribute = attribute
               break

       # Provide informative output
       if problematic_attribute:
           print(f"Identified problematic attribute: {problematic_attribute}")
           print("Potential causes:")
           print("- Attribute contains unpicklable data types (e.g., lambda functions, file objects).")
           print("- Attribute refers to external resources (e.g., database connections).")
           print("- Attribute has circular references within the object's structure.")
       else:
           print("Unable to isolate the specific attribute causing the pickling error.")
           print("Consider:")
           print("- Examining the object's structure and dependencies for potential conflicts.")
           print("- Providing a minimal reproducible example for further analysis.")

   else:
       # If pickling succeeds, indicate no issues found
       print("No pickling issues detected.")


diagnose_pickling_issues(curr_active_pipeline.global_computation_results.computed_data['RankOrder'])

# make a copy of an object

In [ ]:
rank_order_output_path = Path(r'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\2023-12-22_807pm-minimum_inclusion_fr-5-included_qclu_values-[1, 2]RankOrder.pkl').resolve()


from pyphocorehelpers.Filesystem.pickling_helpers import custom_dump, custom_dumps
import dill.detect
dill.detect.trace(True)

# dill.detect.badobjects(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__)

test_obj = curr_active_pipeline.global_computation_results.computed_data['RankOrder'] # pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations.RankOrderComputationsContainer
dill.detect.badtypes(test_obj)

# dill.detect.errors(test_obj)

# with dill.detect.trace(True):
# saveData(rank_order_output_path, (curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__,))
# custom_dumps(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__)
#  with open(pkl_path, file_mode) as dbfile: 
# 	# source, destination
# 	# pickle.dump(db, dbfile)
# 	custom_dump(db, dbfile) # ModuleExcludesPickler
# 	dbfile.close()
	
# # dumps(squared)
# custom_dump(db, dbfile) # ModuleExcludesPickler
# saveData(rank_order_output_path, (curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__,))


In [ ]:
{
	"name": "TypeError",
	"message": "<lambda>() missing 4 required positional arguments: 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', and 'is_forward_replay'",
	"stack": "---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\ReviewOfWork_2024-01-02.ipynb Cell 88 line 1
----> <a href='vscode-notebook-cell:/c%3A/Users/pho/repos/Spike3DWorkEnv/Spike3D/ReviewOfWork_2024-01-02.ipynb#Z2361sZmlsZQ%3D%3D?line=0'>1</a> diagnose_pickling_issues(rank_order_obj.to_dict())

File ~\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\pyphoplacecellanalysis\\General\\Pipeline\\Stages\\ComputationFunctions\\MultiContextComputationFunctions\\RankOrderComputations.py:685, in to_dict(self)
    683 epoch_identifiers = significant_ripple_epochs._df.label.astype({'label': RankOrderAnalyses._label_column_type}).values #.labels
    684 x_values = significant_ripple_epochs.midtimes
--> 685 x_axis_name_suffix = 'Mid-time (Sec)'
    687 significant_ripple_epochs_df = significant_ripple_epochs.to_dataframe()
    688 significant_ripple_epochs_df

File c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\.venv\\lib\\site-packages\\attr\\_next_gen.py:208, in asdict(inst, recurse, filter, value_serializer)
    201 def asdict(inst, *, recurse=True, filter=None, value_serializer=None):
    202     \"\"\"
    203     Same as `attr.asdict`, except that collections types are always retained
    204     and dict is always used as *dict_factory*.
    205 
    206     .. versionadded:: 21.3.0
    207     \"\"\"
--> 208     return _asdict(
    209         inst=inst,
    210         recurse=recurse,
    211         filter=filter,
    212         value_serializer=value_serializer,
    213         retain_collection_types=True,
    214     )

File c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\.venv\\lib\\site-packages\\attr\\_funcs.py:64, in asdict(inst, recurse, filter, dict_factory, retain_collection_types, value_serializer)
     62 if recurse is True:
     63     if has(v.__class__):
---> 64         rv[a.name] = asdict(
     65             v,
     66             recurse=True,
     67             filter=filter,
     68             dict_factory=dict_factory,
     69             retain_collection_types=retain_collection_types,
     70             value_serializer=value_serializer,
     71         )
     72     elif isinstance(v, (tuple, list, set, frozenset)):
     73         cf = v.__class__ if retain_collection_types is True else list

File c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\.venv\\lib\\site-packages\\attr\\_funcs.py:89, in asdict(inst, recurse, filter, dict_factory, retain_collection_types, value_serializer)
     87 elif isinstance(v, dict):
     88     df = dict_factory
---> 89     rv[a.name] = df(
     90         (
     91             _asdict_anything(
     92                 kk,
     93                 is_key=True,
     94                 filter=filter,
     95                 dict_factory=df,
     96                 retain_collection_types=retain_collection_types,
     97                 value_serializer=value_serializer,
     98             ),
     99             _asdict_anything(
    100                 vv,
    101                 is_key=False,
    102                 filter=filter,
    103                 dict_factory=df,
    104                 retain_collection_types=retain_collection_types,
    105                 value_serializer=value_serializer,
    106             ),
    107         )
    108         for kk, vv in v.items()
    109     )
    110 else:
    111     rv[a.name] = v

File c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\.venv\\lib\\site-packages\\attr\\_funcs.py:99, in <genexpr>(.0)
     87 elif isinstance(v, dict):
     88     df = dict_factory
     89     rv[a.name] = df(
     90         (
     91             _asdict_anything(
     92                 kk,
     93                 is_key=True,
     94                 filter=filter,
     95                 dict_factory=df,
     96                 retain_collection_types=retain_collection_types,
     97                 value_serializer=value_serializer,
     98             ),
---> 99             _asdict_anything(
    100                 vv,
    101                 is_key=False,
    102                 filter=filter,
    103                 dict_factory=df,
    104                 retain_collection_types=retain_collection_types,
    105                 value_serializer=value_serializer,
    106             ),
    107         )
    108         for kk, vv in v.items()
    109     )
    110 else:
    111     rv[a.name] = v

File c:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\.venv\\lib\\site-packages\\attr\\_funcs.py:146, in _asdict_anything(val, is_key, filter, dict_factory, retain_collection_types, value_serializer)
    143     else:
    144         cf = list
--> 146     rv = cf(
    147         [
    148             _asdict_anything(
    149                 i,
    150                 is_key=False,
    151                 filter=filter,
    152                 dict_factory=dict_factory,
    153                 retain_collection_types=retain_collection_types,
    154                 value_serializer=value_serializer,
    155             )
    156             for i in val
    157         ]
    158     )
    159 elif isinstance(val, dict):
    160     df = dict_factory

TypeError: <lambda>() missing 4 required positional arguments: 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', and 'is_forward_replay'"
}

In [ ]:
print(list(rank_order_obj.__dict__.keys())) # ['is_global', 'LR_ripple', 'RL_ripple', 'LR_laps', 'RL_laps', 'ripple_most_likely_result_tuple', 'laps_most_likely_result_tuple', 'ripple_combined_epoch_stats_df', 'ripple_new_output_tuple', 'laps_combined_epoch_stats_df', 'laps_new_output_tuple', 'minimum_inclusion_fr_Hz', 'included_qclu_values']
for k, v in rank_order_obj.__dict__.items():
    print(f'trying to pickle: {k}')
    try:
        diagnose_pickling_issues(v)
    except TypeError as e:
        print(f'failed to pickle .{k} with error {e}')

In [ ]:
['is_global', 'LR_ripple', 'RL_ripple', 'LR_laps', 'RL_laps', 'ripple_most_likely_result_tuple', 'laps_most_likely_result_tuple', 'ripple_combined_epoch_stats_df', 'ripple_new_output_tuple', 'laps_combined_epoch_stats_df', 'laps_new_output_tuple', 'minimum_inclusion_fr_Hz', 'included_qclu_values']
trying to pickle: is_global
No pickling issues detected.
trying to pickle: LR_ripple
failed to pickle .LR_ripple with error cannot pickle 'QApplication' object
trying to pickle: RL_ripple
failed to pickle .RL_ripple with error cannot pickle 'QApplication' object
trying to pickle: LR_laps
failed to pickle .LR_laps with error cannot pickle 'QApplication' object
trying to pickle: RL_laps
failed to pickle .RL_laps with error cannot pickle 'QApplication' object
trying to pickle: ripple_most_likely_result_tuple
No pickling issues detected.
trying to pickle: laps_most_likely_result_tuple
No pickling issues detected.
trying to pickle: ripple_combined_epoch_stats_df
No pickling issues detected.
trying to pickle: ripple_new_output_tuple
No pickling issues detected.
trying to pickle: laps_combined_epoch_stats_df
No pickling issues detected.
trying to pickle: laps_new_output_tuple
No pickling issues detected.
trying to pickle: minimum_inclusion_fr_Hz
No pickling issues detected.
trying to pickle: included_qclu_values
No pickling issues detected.

In [ ]:
['is_global', 'LR_ripple', 'RL_ripple', 'LR_laps', 'RL_laps', 'ripple_most_likely_result_tuple', 'laps_most_likely_result_tuple', 'ripple_combined_epoch_stats_df', 'ripple_new_output_tuple', 'laps_combined_epoch_stats_df', 'laps_new_output_tuple', 'minimum_inclusion_fr_Hz', 'included_qclu_values']
trying to pickle: .is_global
No pickling issues detected.
trying to pickle: .LR_ripple
Pickling error encountered: cannot pickle 'QApplication' object
Identified problematic attribute: to_hdf
Potential causes:
- Attribute contains unpicklable data types (e.g., lambda functions, file objects).
- Attribute refers to external resources (e.g., database connections).
- Attribute has circular references within the object's structure.
trying to pickle: .RL_ripple
Pickling error encountered: cannot pickle 'QApplication' object
Identified problematic attribute: to_hdf
Potential causes:
- Attribute contains unpicklable data types (e.g., lambda functions, file objects).
- Attribute refers to external resources (e.g., database connections).
- Attribute has circular references within the object's structure.
trying to pickle: .LR_laps
Pickling error encountered: cannot pickle 'QApplication' object
Identified problematic attribute: to_hdf
Potential causes:
- Attribute contains unpicklable data types (e.g., lambda functions, file objects).
- Attribute refers to external resources (e.g., database connections).
- Attribute has circular references within the object's structure.
trying to pickle: .RL_laps
Pickling error encountered: cannot pickle 'QApplication' object
Identified problematic attribute: to_hdf
Potential causes:
- Attribute contains unpicklable data types (e.g., lambda functions, file objects).
- Attribute refers to external resources (e.g., database connections).
- Attribute has circular references within the object's structure.
trying to pickle: .ripple_most_likely_result_tuple
No pickling issues detected.
trying to pickle: .laps_most_likely_result_tuple
No pickling issues detected.
trying to pickle: .ripple_combined_epoch_stats_df
No pickling issues detected.
trying to pickle: .ripple_new_output_tuple
No pickling issues detected.
trying to pickle: .laps_combined_epoch_stats_df
No pickling issues detected.
trying to pickle: .laps_new_output_tuple
No pickling issues detected.
trying to pickle: .minimum_inclusion_fr_Hz
No pickling issues detected.
trying to pickle: .included_qclu_values
No pickling issues detected.


In [ ]:
# Clear issue in `rank_order_obj.LR_ripple.ranked_aclus_stats_dict`: values contain LongShortStatsTuple which contains long_stats_z_scorer
`pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations.Zscorer`

In [ ]:
['is_global', 'ranked_aclus_stats_dict', 'selected_spikes_fragile_linear_neuron_IDX_dict', 'long_z_score', 'short_z_score', 'long_short_z_score_diff', 'spikes_df', 'epochs_df', 'selected_spikes_df', 'extra_info_dict']
trying to pickle: .is_global
		No pickling issues detected.

trying to pickle: .ranked_aclus_stats_dict
		Pickling error encountered: Can't pickle <function make_set_closure_cell.<locals>.set_closure_cell at 0x000002D510CD7820>: it's not found as attr._compat.make_set_closure_cell.<locals>.set_closure_cell
Identified problematic attribute: values
Potential causes:
- Attribute contains unpicklable data types (e.g., lambda functions, file objects).
- Attribute refers to external resources (e.g., database connections).
- Attribute has circular references within the object's structure.

trying to pickle: .selected_spikes_fragile_linear_neuron_IDX_dict
		No pickling issues detected.

trying to pickle: .long_z_score
		No pickling issues detected.

trying to pickle: .short_z_score
		No pickling issues detected.

trying to pickle: .long_short_z_score_diff
		No pickling issues detected.

trying to pickle: .spikes_df
		No pickling issues detected.

trying to pickle: .epochs_df
		No pickling issues detected.

trying to pickle: .selected_spikes_df
		No pickling issues detected.

trying to pickle: .extra_info_dict
		No pickling issues detected.

